In [1]:
import pandas as pd
# ensure that all columns are shown and that colum content is not cut
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width',1000)
pd.set_option('display.max_rows', 500) # ensure that all rows are shown

# Bulk Data Processing Deep Dive
The main advantage of this library is that the all data is downloaded down to your computer and therefore makes it easy to analyze all the data at once. 

For instance, if you want to implement your own screener.

Just on the file system, the size of all parquet files is more than 5 GB and growing with every quarter of new data. Since the parquet format is also storage optimized, loading all the data into memory would need significantly more memory than a standard computer/laptop usually provides.

Hence it is important to filter the data during the loading process, so that you only load the data into memory that is really needed.

**NOTE**: with the new automation feature (introduced with version 1.8.0), it is possible to add processing steps directly to the normal update process. That means, whenever a new file is detected on SEC.gov, not only downloading, transforming into parquet, and indexing it can be executed, but also customized additional processing steps - like the steps described in this notebook which filters the data and produces files containing all the data. For an example, that directly can be used and shows how it works, have a look at the 08_00_automation_bascis notebook.

## Prepare Datasets
In the first part of this notebook, we will create four different datasets for all the balance sheet datapoints, the cashflow datapoints, the income statement datapoints, and the cover page datapoints.
These datasets will be stored in their own directories, so that they can be easily loaded afterwards. Moreover, we will store the raw version (where the num_df and the pre_df are not joined) and the joined version, where num_df and pre_df are joined. Depending on what you want to do/analyze, you can use either one.

**Note:** The code that is explained here is also available in the modul `bulk_loading` which is inside the `secfsdstools.u_usecase` package.

This notebook will show two approaches. The first one is loading all the data in parallel, which you can do if you have enough resources in your computer. The second is doing it sequentially, which is slower, but needs less memory. In the end, you will create these datasets once and extend it when new quarterly zip files arrive, or you will recreate them once every quarter. So in the end it doesn't really matter if the process takes 15 minutes or an hour.

We will also apply different filters:

* only filter 10-K and 10-Q reports during loading
* `ReportPeriodRawFilter`: since we are only interested in datapoints that belong to the period of the report
* `MainCoregRawFilter`: since we don't want to see datapoints of a subsidiary
* `OfficialTagsOnlyRawFilter`: since we want to be able to compare the content and therefore don't want to read tags that or not in the standard sec xbrl definition
* `USDOnlyRawFilter`: since we are not interested in money datapoints that are not in USD

### Basics
First, we will defines some basic stuff that is used by both approaches.

In [2]:
import os
from secfsdstools.d_container.databagmodel import RawDataBag, JoinedDataBag
from secfsdstools.e_collector.zipcollecting import ZipCollector

2025-02-03 06:20:18,881 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg


The following list defines which statements we want to load.

In [3]:
statements_to_load = ["BS", "CF", "IS", "CP"]

Next, we define a filter function, that defines the whole chain. As mentioned in the 04_collector_deep_dive.ipynt notebook, we have to define the imports inside the function itself, if we want to use it in jupyter together with parallization.

In [4]:
def postloadfilter(databag: RawDataBag) -> RawDataBag:
    from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, MainCoregRawFilter, OfficialTagsOnlyRawFilter, USDOnlyRawFilter

    return databag[ReportPeriodRawFilter()][MainCoregRawFilter()][OfficialTagsOnlyRawFilter()][USDOnlyRawFilter()]

Next is a simple function that takes a raw databag and creates the joined databag. Both, the rawdatabag and the joined databag are then stored in a specific folder.


In [5]:
def save_databag(databag: RawDataBag, financial_statement: str, base_path: str) -> JoinedDataBag:
    target_path_raw = os.path.join(base_path, financial_statement, 'raw')
    print(f"store rawdatabag under {target_path_raw}")
    os.makedirs(target_path_raw, exist_ok=True)
    databag.save(target_path_raw)
    
    target_path_joined = os.path.join(base_path, financial_statement, 'joined')
    os.makedirs(target_path_joined, exist_ok=True)
    print("create joined databag")
    joined_databag = databag.join()
    
    print(f"store joineddatabag under {target_path_joined}")
    joined_databag.save(target_path_joined)
    return joined_databag

### Parallel Data Loading
As stated above, we want to load all available 10-K and 10-Q reports. Therefore, we can use the `ZipCollector`which provides an option to load data from all available zip files. 

Moreover, the implementation of the ziploader uses all your cores in order to load data from your disk into memory. So you don't have to implement the parallization yourself. There are 50+ zip files that have to be loaded, so if you have 4 cores, you will load 4 at one time.

Also, the `ZipCollector` provides parameters for filtering the report type (10-K and 10-Q) amd the financial statement type (Balance Sheet, Casch Flow, or Income Statement). These filters are directly applied during loading, since the data is stored in Parquet format. This will already reduce that amount of data that is being loaded into memory significantly.

Moreover, it also provides the post_load_filter which we can use to apply the other filters, defined in the postloadfilter function.

In [7]:
def load_all_financial_statements_parallel(financial_statement: str) -> RawDataBag:
    """ 
    financial_statement: either "BS", "CF", or "IS"
    """

    collector: ZipCollector = ZipCollector.get_all_zips(forms_filter=["10-K", "10-Q"],
                                                        stmt_filter=[financial_statement],
                                                        post_load_filter=postloadfilter)
    return collector.collect()

We loop over the statements that we want to load and collect their datapoints into a specific dataset.

This process will take several minutes. On my laptop the execution time was approximately 30 minutes (32GB Ram / 4/8 Cores).

In [8]:
for statement_to_load in statements_to_load:
    rawdatabag = load_all_financial_statements_parallel(financial_statement=statement_to_load)
    save_databag(databag=rawdatabag, financial_statement=statement_to_load, base_path="./set/parallel/")

2025-02-02 12:57:51,516 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 12:57:51,521 [INFO] parallelexecution      items to process: 63
2025-02-02 13:03:12,884 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under ./set/parallel/BS\raw
create joined databag
store joineddatabag under ./set/parallel/BS\joined


2025-02-02 13:05:41,117 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 13:05:41,125 [INFO] parallelexecution      items to process: 63
2025-02-02 13:11:07,236 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under ./set/parallel/CF\raw
create joined databag
store joineddatabag under ./set/parallel/CF\joined


2025-02-02 13:13:25,329 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 13:13:25,342 [INFO] parallelexecution      items to process: 63
2025-02-02 13:18:26,399 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under ./set/parallel/IS\raw
create joined databag
store joineddatabag under ./set/parallel/IS\joined


2025-02-02 13:20:40,704 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 13:20:40,712 [INFO] parallelexecution      items to process: 63
2025-02-02 13:25:13,782 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under ./set/parallel/CP\raw
create joined databag
store joineddatabag under ./set/parallel/CP\joined


After processing, you have the following structure and sizes (with data up to 2024 Q4):
<pre>
- set/parallel
  - BS
    - raw     : 1.44 GB
    - joined  :  650 MB
  - CF
    - raw     : 1.43 GB
    - joined  :  426 MB
  - IS
    - raw     : 1.36 GB
    - joined  :  483 MB
  - CP
    - raw     : 1.26 GB
    - joined  :   19 MB    
</pre>

Especially the joined databags have a size that can be easily loaded. Moreover, loading them just takes a few seconds. 

In [9]:
#load BS joined data
joinedBS = JoinedDataBag.load("./set/parallel/BS/joined")
print("loaded BS databag: ", joinedBS.pre_num_df.shape)
joinedCF = JoinedDataBag.load("./set/parallel/CF/joined")
print("loaded CF databag: ", joinedCF.pre_num_df.shape)
joinedIS = JoinedDataBag.load("./set/parallel/IS/joined")
print("loaded IS databag: ", joinedIS.pre_num_df.shape)
joinedCP = JoinedDataBag.load("./set/parallel/CP/joined")
print("loaded CP databag: ", joinedCP.pre_num_df.shape)

loaded BS databag:  (19657047, 17)
loaded CF databag:  (12883947, 17)
loaded IS databag:  (17412439, 17)
loaded CP databag:  (76, 17)


### Serial Data Loading
As mentioned above, parallel loading requires some minimal ressources on your laptop/computer. However, using a serial process, you still can create the databags for all balance sheet, cash flow, and income statments. Of course, we need more code and we will also save intermediate results on disk.

The first thing which we need, is a list of all available zip-files. Actually, we just can copy the code from `ZipCollector.get_all_zips()`.

In [6]:
from typing import List
from secfsdstools.a_config.configmgt import ConfigurationManager
from secfsdstools.c_index.indexdataaccess import ParquetDBIndexingAccessor

def read_all_zip_names() -> List[str]:
    configuration = ConfigurationManager.read_config_file()
    dbaccessor = ParquetDBIndexingAccessor(db_dir=configuration.db_dir)

    # exclude 2009q1.zip, since this is empty and causes an error when it is read with a filter
    filenames = [x.fileName for x in dbaccessor.read_all_indexfileprocessing() if not x.fullPath.endswith("2009q1.zip")]
    return filenames

In [7]:
all_zip_names = read_all_zip_names()
print(len(all_zip_names))
print(all_zip_names)

2025-02-02 14:22:59,558 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg


63
['2012q4.zip', '2013q3.zip', '2018q2.zip', '2024q1.zip', '2015q1.zip', '2018q4.zip', '2019q1.zip', '2014q2.zip', '2012q1.zip', '2022q3.zip', '2016q3.zip', '2023q1.zip', '2024q3.zip', '2015q2.zip', '2015q3.zip', '2014q3.zip', '2013q2.zip', '2018q1.zip', '2014q1.zip', '2009q4.zip', '2019q2.zip', '2022q4.zip', '2011q1.zip', '2010q2.zip', '2017q1.zip', '2020q1.zip', '2016q1.zip', '2020q2.zip', '2024q4.zip', '2009q2.zip', '2021q3.zip', '2017q3.zip', '2021q4.zip', '2020q4.zip', '2011q2.zip', '2010q3.zip', '2014q4.zip', '2021q2.zip', '2022q2.zip', '2016q4.zip', '2019q3.zip', '2013q4.zip', '2010q1.zip', '2024q2.zip', '2022q1.zip', '2023q3.zip', '2009q3.zip', '2010q4.zip', '2018q3.zip', '2023q2.zip', '2017q2.zip', '2012q3.zip', '2011q3.zip', '2016q2.zip', '2012q2.zip', '2023q4.zip', '2019q4.zip', '2015q4.zip', '2021q1.zip', '2011q4.zip', '2013q1.zip', '2020q3.zip', '2017q4.zip']


**Prepare the temporary dataset**
Next, prepare the data for every single zip-file. So for every zip-file, we collect the datapoints for BS, CF, and IS and apply the aove defined filters. The following functions takes care of that.

In [8]:
def build_tmp_set(financial_statement: str, file_names: List[str], target_path: str = "set/tmp/"):
    """ This function reads the data in sequence from the provided list of zip file names. It filters according to the 
        defined financial_statement and stores the data in specific subfolders.
        
        the folder structure will look like
        <target_path>/<file_name>/<financial_statement>/raw
        <target_path>/<file_name>/<financial_statement>/joined                                       
        """
    
    for file_name in file_names:
        collector = ZipCollector.get_zip_by_name(name=file_name,
                                 forms_filter=["10-K", "10-Q"],
                                 stmt_filter=[financial_statement],
                                 post_load_filter=postloadfilter)

        rawdatabag = collector.collect()

        base_path = os.path.join(target_path, file_name)
        # saving the raw databag, joining and saving the joined databag
        save_databag(databag=rawdatabag, financial_statement=financial_statement, base_path=base_path)

We call the function for every statement (BS, CF, IS, and CP).
As a reference, running all four cells took about 15 minutes on my laptop (32GB Ram / 4/8 Cores)

In [9]:
build_tmp_set(financial_statement="BS", file_names=all_zip_names, target_path="set/tmp/")

2025-02-02 14:23:37,097 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:23:37,102 [INFO] parallelexecution      items to process: 1
2025-02-02 14:23:37,103 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2012q4.zip
2025-02-02 14:23:40,307 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2012q4.zip\BS\raw
create joined databag
store joineddatabag under set/tmp/2012q4.zip\BS\joined


2025-02-02 14:23:42,806 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:23:42,809 [INFO] parallelexecution      items to process: 1
2025-02-02 14:23:42,810 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2013q3.zip
2025-02-02 14:23:45,864 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2013q3.zip\BS\raw
create joined databag
store joineddatabag under set/tmp/2013q3.zip\BS\joined


2025-02-02 14:23:48,467 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:23:48,470 [INFO] parallelexecution      items to process: 1
2025-02-02 14:23:48,470 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2018q2.zip
2025-02-02 14:23:51,147 [INFO] parallelexecution      commited chunk: 0


...

store rawdatabag under set/tmp/2013q1.zip\BS\raw
create joined databag
store joineddatabag under set/tmp/2013q1.zip\BS\joined


2025-02-02 14:28:52,390 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:28:52,394 [INFO] parallelexecution      items to process: 1
2025-02-02 14:28:52,396 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2020q3.zip
2025-02-02 14:28:55,757 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2020q3.zip\BS\raw
create joined databag
store joineddatabag under set/tmp/2020q3.zip\BS\joined


2025-02-02 14:28:58,635 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:28:58,638 [INFO] parallelexecution      items to process: 1
2025-02-02 14:28:58,639 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2017q4.zip
2025-02-02 14:29:01,600 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2017q4.zip\BS\raw
create joined databag
store joineddatabag under set/tmp/2017q4.zip\BS\joined


In [10]:
build_tmp_set(financial_statement="CF", file_names=all_zip_names, target_path="set/tmp/")

2025-02-02 14:29:04,489 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:29:04,493 [INFO] parallelexecution      items to process: 1
2025-02-02 14:29:04,495 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2012q4.zip
2025-02-02 14:29:07,524 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2012q4.zip\CF\raw
create joined databag
store joineddatabag under set/tmp/2012q4.zip\CF\joined


2025-02-02 14:29:09,965 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:29:09,968 [INFO] parallelexecution      items to process: 1
2025-02-02 14:29:09,969 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2013q3.zip
2025-02-02 14:29:12,869 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2013q3.zip\CF\raw
create joined databag
store joineddatabag under set/tmp/2013q3.zip\CF\joined


2025-02-02 14:29:15,262 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:29:15,265 [INFO] parallelexecution      items to process: 1
2025-02-02 14:29:15,266 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2018q2.zip
2025-02-02 14:29:17,703 [INFO] parallelexecution      commited chunk: 0


...

store rawdatabag under set/tmp/2013q1.zip\CF\raw
create joined databag
store joineddatabag under set/tmp/2013q1.zip\CF\joined


2025-02-02 14:33:40,286 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:33:40,289 [INFO] parallelexecution      items to process: 1
2025-02-02 14:33:40,290 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2020q3.zip
2025-02-02 14:33:43,196 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2020q3.zip\CF\raw
create joined databag
store joineddatabag under set/tmp/2020q3.zip\CF\joined


2025-02-02 14:33:45,372 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:33:45,375 [INFO] parallelexecution      items to process: 1
2025-02-02 14:33:45,376 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2017q4.zip
2025-02-02 14:33:47,945 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2017q4.zip\CF\raw
create joined databag
store joineddatabag under set/tmp/2017q4.zip\CF\joined


In [11]:
build_tmp_set(financial_statement="IS", file_names=all_zip_names, target_path="set/tmp/")

2025-02-02 14:33:50,352 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:33:50,355 [INFO] parallelexecution      items to process: 1
2025-02-02 14:33:50,356 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2012q4.zip
2025-02-02 14:33:53,036 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2012q4.zip\IS\raw
create joined databag
store joineddatabag under set/tmp/2012q4.zip\IS\joined


2025-02-02 14:33:55,379 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:33:55,382 [INFO] parallelexecution      items to process: 1
2025-02-02 14:33:55,387 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2013q3.zip
2025-02-02 14:33:58,172 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2013q3.zip\IS\raw
create joined databag
store joineddatabag under set/tmp/2013q3.zip\IS\joined


2025-02-02 14:34:00,636 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:34:00,639 [INFO] parallelexecution      items to process: 1
2025-02-02 14:34:00,640 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2018q2.zip
2025-02-02 14:34:03,009 [INFO] parallelexecution      commited chunk: 0


...

store rawdatabag under set/tmp/2013q1.zip\IS\raw
create joined databag
store joineddatabag under set/tmp/2013q1.zip\IS\joined


2025-02-02 14:38:15,580 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:38:15,583 [INFO] parallelexecution      items to process: 1
2025-02-02 14:38:15,584 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2020q3.zip
2025-02-02 14:38:18,485 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2020q3.zip\IS\raw
create joined databag
store joineddatabag under set/tmp/2020q3.zip\IS\joined


2025-02-02 14:38:20,908 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:38:20,911 [INFO] parallelexecution      items to process: 1
2025-02-02 14:38:20,912 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2017q4.zip
2025-02-02 14:38:23,468 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2017q4.zip\IS\raw
create joined databag
store joineddatabag under set/tmp/2017q4.zip\IS\joined


In [12]:
build_tmp_set(financial_statement="CP", file_names=all_zip_names, target_path="set/tmp/")

2025-02-02 14:38:25,875 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:38:25,879 [INFO] parallelexecution      items to process: 1
2025-02-02 14:38:25,880 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2012q4.zip
2025-02-02 14:38:28,721 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2012q4.zip\CP\raw
create joined databag


2025-02-02 14:38:30,232 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:38:30,235 [INFO] parallelexecution      items to process: 1
2025-02-02 14:38:30,236 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2013q3.zip


store joineddatabag under set/tmp/2012q4.zip\CP\joined


2025-02-02 14:38:32,905 [INFO] parallelexecution      commited chunk: 0


...

store rawdatabag under set/tmp/2020q3.zip\CP\raw


2025-02-02 14:42:05,166 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 14:42:05,169 [INFO] parallelexecution      items to process: 1
2025-02-02 14:42:05,170 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2017q4.zip


create joined databag
store joineddatabag under set/tmp/2020q3.zip\CP\joined


2025-02-02 14:42:07,623 [INFO] parallelexecution      commited chunk: 0


store rawdatabag under set/tmp/2017q4.zip\CP\raw
create joined databag
store joineddatabag under set/tmp/2017q4.zip\CP\joined


We know have subfolders for BS, CF, IS, and CP for every quarterly zipfile with the corresponding datapoints.

**Create the rawdatabags**

In [13]:
from glob import glob

def create_rawdatabag(financial_statement: str, target_path: str):
    raw_files = glob(f"./set/tmp/*/{financial_statement}/raw/", recursive = True)    
    raw_databags = [RawDataBag.load(file) for file in raw_files]
    raw_databag = RawDataBag.concat(raw_databags)
    target_path_raw = os.path.join(target_path, financial_statement, 'raw')
    print(f"store rawdatabag under {target_path_raw}")
    os.makedirs(target_path_raw, exist_ok=True)
    raw_databag.save(target_path_raw)      

Next, concatenate the raw datasets together. Again, as a reference it took about 6-7 minutes to create all four rawdatabags.

In [14]:
create_rawdatabag(financial_statement="BS", target_path="set/serial/")

store rawdatabag under set/serial/BS\raw


In [15]:
create_rawdatabag(financial_statement="CF", target_path="set/serial/")

store rawdatabag under set/serial/CF\raw


In [16]:
create_rawdatabag(financial_statement="IS", target_path="set/serial/")

store rawdatabag under set/serial/IS\raw


In [17]:
create_rawdatabag(financial_statement="CP", target_path="set/serial/")

store rawdatabag under set/serial/CP\raw


**Create the joined databags**

In [18]:
from glob import glob

def create_joineddatabag(financial_statement: str, target_path: str):
    joined_files = glob(f"./set/tmp/*/{financial_statement}/joined/", recursive = True)
    joined_databags = [JoinedDataBag.load(file) for file in joined_files]
    joined_databag = JoinedDataBag.concat(joined_databags)
    target_path_joined = os.path.join(target_path, financial_statement, 'joined')
    print(f"store joineddatabag under {target_path_joined}")
    os.makedirs(target_path_joined, exist_ok=True)
    joined_databag.save(target_path_joined)   

Finally, create the joined databags. To create all four datasets, it took about 90 seconds.

In [19]:
create_joineddatabag(financial_statement="BS", target_path="set/serial/")

store joineddatabag under set/serial/BS\joined


In [20]:
create_joineddatabag(financial_statement="CF", target_path="set/serial/")

store joineddatabag under set/serial/CF\joined


In [21]:
create_joineddatabag(financial_statement="IS", target_path="set/serial/")

store joineddatabag under set/serial/IS\joined


In [22]:
create_joineddatabag(financial_statement="CP", target_path="set/serial/")

store joineddatabag under set/serial/CP\joined


Now we can read back all four prepared joined datasets. This only takes a few seconds.

In [14]:
#load BS joined data
joinedBS = JoinedDataBag.load("./set/serial/BS/joined")
print("loaded BS databag: ", joinedBS.pre_num_df.shape)
joinedCF = JoinedDataBag.load("./set/serial/CF/joined")
print("loaded CF databag: ", joinedCF.pre_num_df.shape)
joinedIS = JoinedDataBag.load("./set/serial/IS/joined")
print("loaded IS databag: ", joinedIS.pre_num_df.shape)
joinedCP = JoinedDataBag.load("./set/serial/CP/joined")
print("loaded CP databag: ", joinedCP.pre_num_df.shape)

loaded BS databag:  (19657047, 17)
loaded CF databag:  (12883947, 17)
loaded IS databag:  (17412439, 17)
loaded CP databag:  (76, 17)
